Урок 4. Изучение Spark Structure Streaming

Условие: используйте источник rate, напишите код, который создаст дополнительный столбец, который будет выводить сумму только нечётных чисел.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

In [64]:
spark = SparkSession.builder.appName("SumOfOddNumbers").getOrCreate()

# Подключаем тестовый источник данных "rate"
streaming_df = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# Фильтруем поступающие данные на нечетность
odd_numbers_df = streaming_df.filter("value % 2 != 0")

# Суммируем значения
odd_sum_df = odd_numbers_df.groupBy().agg(sum("value").alias("sum_of_odds"))

# Выводим данные (в Google Colab не выводится в консоль, делаем через промежуточную таблицу)
query = odd_sum_df.writeStream.outputMode("update").queryName("output_data").format("memory").start()

# Ждём завершения потока
query.awaitTermination(20)
query.stop()

# Выводим данные промежуточной таблицы (убираем NULL первого подсчета суммы и дубли значений суммы от четных элементов)
spark.sql("SELECT DISTINCT(sum_of_odds) FROM output_data WHERE sum_of_odds IS NOT NULL ORDER BY sum_of_odds").show(truncate=False)
spark.stop()

+-----------+
|sum_of_odds|
+-----------+
|1          |
|4          |
|9          |
|16         |
|25         |
|36         |
|49         |
|64         |
|81         |
+-----------+

